In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
warnings.filterwarnings("ignore")
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

In [ ]:
data = pd.read_csv('Crime_Data_from_2020_to_Present.csv')

data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
#Datetime

In [ ]:
for x in ['Date Rptd', 'DATE OCC']:
    data[x] = pd.to_datetime(data[x])

In [ ]:
#Vict Sex

In [ ]:
data['Vict Sex'].unique()

In [ ]:
data['Vict Sex'] = data['Vict Sex'].str.replace('X', 'Unknown')
data['Vict Sex'] = data['Vict Sex'].str.replace('-', 'Unknown')
data['Vict Sex'] = data['Vict Sex'].str.replace('H', 'Unknown')

In [ ]:
data['Vict Sex'].fillna('Unknown', inplace=True)

In [ ]:
#Age

In [ ]:
data['Vict Age'].unique()

In [ ]:
data['Vict Age'] = data['Vict Age'].replace(0, round(data['Vict Age'].mean()))

In [ ]:
for x in data['Vict Age']:
    if x < 0:
        data['Vict Age'] = data['Vict Age'].replace(x, round(data['Vict Age'].mean()))

In [ ]:
#Vict Descent

In [ ]:
data['Vict Descent'].unique()

In [ ]:
data['Vict Descent'].fillna('Unknown', inplace=True)

In [ ]:
def race(data):
    blanks = []
    for x in data['Vict Descent']:
        if x == 'B':
            blanks.append('Black')
        elif x == 'H':
            blanks.append('Hispanic/Latin/Mexican')
        elif x == 'X':
            blanks.append('Unknown')
        elif x == '-':
            blanks.append('Unknown')
        elif x == 'W':
            blanks.append('White')
        elif x == 'A':
            blanks.append('Asian')
        elif x == 'O':
            blanks.append('Other')
        elif x == 'C':
            blanks.append('Chinese')
        elif x == 'F':
            blanks.append('Filipino')
        elif x == 'K':
            blanks.append('Korean')
        elif x == 'I':
            blanks.append('American Indian/Alaskan Native')
        elif x == 'V':
            blanks.append('Vietnamese')
        elif x == 'Z':
            blanks.append('Asian Indian')
        elif x == 'J':
            blanks.append('Japanese')
        elif x == 'P':
            blanks.append('Pacific Islanders')
        elif x == 'S':
            blanks.append('Samoan')
        elif x == 'G':
            blanks.append('Guamanian')
        elif x == 'U':
            blanks.append('Hawaiian')
        elif x == 'D':
            blanks.append('Cambodian')
        elif x == 'L':
            blanks.append('Laotian')
        else:
            blanks.append('Unknown')
            
    return blanks

In [ ]:
data['Vict Descent'] = race(data)

In [ ]:
#Mocodes

In [ ]:
data['Mocodes'].fillna('Unknown', inplace=True)

In [ ]:
data.fillna('Unknown', inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
#Crime DESC categorization (chatgpt)

In [ ]:
crime_category_dict = {
    'BATTERY - SIMPLE ASSAULT': 'Violent Crimes',
    'SEX OFFENDER REGISTRANT OUT OF COMPLIANCE': 'Sex Crimes',
    'VANDALISM - MISDEMEANOR ($399 OR UNDER)': 'Property Crimes',
    'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)': 'Property Crimes',
    'RAPE, FORCIBLE': 'Violent Crimes',
    'SHOPLIFTING - PETTY THEFT ($950 & UNDER)': 'Property Crimes',
    'OTHER MISCELLANEOUS CRIME': 'Crimes Against Persons or Public Order',
    'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD': 'Property Crimes',
    'BURGLARY FROM VEHICLE': 'Property Crimes',
    'CRIMINAL THREATS - NO WEAPON DISPLAYED': 'Crimes Against Persons or Public Order',
    'ARSON': 'Crimes Against Persons or Public Order',
    'INTIMATE PARTNER - SIMPLE ASSAULT': 'Crimes Against Persons or Public Order',
    'THEFT PLAIN - PETTY ($950 & UNDER)': 'Property Crimes',
    'THEFT OF IDENTITY': 'Property Crimes',
    'ROBBERY': 'Violent Crimes',
    'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT': 'Violent Crimes',
    'BURGLARY': 'Property Crimes',
    'VEHICLE - STOLEN': 'Crimes Against Property and Vehicles',
    'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)': 'Property Crimes',
    'BRANDISH WEAPON': 'Crimes Involving Weapons',
    'INTIMATE PARTNER - AGGRAVATED ASSAULT': 'Violent Crimes',
    'BUNCO, GRAND THEFT': 'Financial Crimes',
    'THEFT, PERSON': 'Property Crimes',
    'BATTERY WITH SEXUAL CONTACT': 'Sex Crimes',
    'BIKE - STOLEN': 'Crimes Against Property and Vehicles',
    'BATTERY POLICE (SIMPLE)': 'Crimes Against Law Enforcement',
    'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD': 'Crimes Against Persons or Public Order',
    'VIOLATION OF COURT ORDER': 'Crimes Against Persons or Public Order',
    'TRESPASSING': 'Crimes Against Persons or Public Order',
    'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)': 'Property Crimes',
    'VIOLATION OF RESTRAINING ORDER': 'Crimes Against Persons or Public Order',
    'DISTURBING THE PEACE': 'Crimes Against Persons or Public Order',
    'THEFT FROM MOTOR VEHICLE - ATTEMPT': 'Property Crimes',
    'THROWING OBJECT AT MOVING VEHICLE': 'Crimes Against Persons or Public Order',
    'EXTORTION': 'Crimes Against Persons or Public Order',
    'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ': 'Sex Crimes',
    'CHILD STEALING': 'Crimes Against Children',
    'CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 YRS OLDER)': 'Crimes Against Children',
    'ATTEMPTED ROBBERY': 'Violent Crimes',
    'OTHER ASSAULT': 'Crimes Against Persons or Public Order',
    'BOMB SCARE': 'Crimes Against Persons or Public Order',
    'DOCUMENT FORGERY / STOLEN FELONY': 'Financial Crimes',
    'SEXUAL PENETRATION W/FOREIGN OBJECT': 'Sexual Offenses',
    'SHOTS FIRED AT INHABITED DWELLING': 'Crimes Against Persons or Public Order',
    'BURGLARY, ATTEMPTED': 'Property Crimes',
    'INDECENT EXPOSURE': 'Crimes Against Persons or Public Order',
    'ORAL COPULATION': 'Sex Crimes',
    'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)': 'Financial Crimes',
    'VIOLATION OF TEMPORARY RESTRAINING ORDER': 'Crimes Against Persons or Public Order',
    'BUNCO, PETTY THEFT': 'Financial Crimes',
    'KIDNAPPING - GRAND ATTEMPT': 'Violent Crimes',
    'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)': 'Property Crimes',
    'RESISTING ARREST': 'Crimes Against Persons or Public Order',
    'DISCHARGE FIREARMS/SHOTS FIRED': 'Crimes Against Persons or Public Order',
    'THREATENING PHONE CALLS/LETTERS': 'Crimes Against Persons or Public Order',
    'KIDNAPPING': 'Violent Crimes',
    'LEWD/LASCIVIOUS ACTS WITH CHILD': 'Sex Crimes',
    'LEWD CONDUCT': 'Crimes Against Persons or Public Order',
    'UNAUTHORIZED COMPUTER ACCESS': 'Financial Crimes',
    'PURSE SNATCHING': 'Property Crimes',
    'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH': 'Sexual Offenses',
    'CHILD NEGLECT (SEE 300 W.I.C.)': 'Crimes Against Children',
    'CONTEMPT OF COURT': 'Crimes Against Persons or Public Order',
    'CHILD ANNOYING (17YRS & UNDER)': 'Crimes Against Children',
    'BUNCO, ATTEMPT': 'Financial Crimes',
    'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT': 'Crimes Against Children',
    'PIMPING': 'Financial Crimes',
    'STALKING': 'Crimes Against Persons or Public Order',
    'THEFT PLAIN - ATTEMPT': 'Property Crimes',
    'RAPE, ATTEMPTED': 'Violent Crimes',
    'SHOPLIFTING - ATTEMPT': 'Property Crimes',
    'THEFT FROM PERSON - ATTEMPT': 'Property Crimes',
    'FALSE IMPRISONMENT': 'Crimes Against Persons or Public Order',
    'BURGLARY FROM VEHICLE, ATTEMPTED': 'Property Crimes',
    'FAILURE TO YIELD': 'Crimes Against Persons or Public Order',
    'VEHICLE - ATTEMPT STOLEN': 'Crimes Against Property and Vehicles',
    'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)': 'Financial Crimes',
    'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 & UNDER': 'Financial Crimes',
    'COUNTERFEIT': 'Financial Crimes',
    'CREDIT CARDS, FRAUD USE ($950 & UNDER': 'Financial Crimes',
    'PICKPOCKET': 'Property Crimes',
    'SHOTS FIRED AT MOVING VEHICLE, TRAIN OR AIRCRAFT': 'Crimes Against Persons or Public Order',
    'CRIMINAL HOMICIDE': 'Violent Crimes',
    'PROWLER': 'Crimes Against Persons or Public Order',
    'DOCUMENT WORTHLESS ($200 & UNDER)': 'Financial Crimes',
    'DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $950.01': 'Financial Crimes',
    'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER': 'Crimes Against Law Enforcement',
    'DISHONEST EMPLOYEE - GRAND THEFT': 'Financial Crimes',
    'HUMAN TRAFFICKING - COMMERCIAL SEX ACTS': 'Crimes Against Persons or Public Order',
    'CHILD PORNOGRAPHY': 'Sex Crimes',
    'PEEPING TOM': 'Crimes Against Persons or Public Order',
    'BATTERY ON A FIREFIGHTER': 'Crimes Against Law Enforcement',
    'TILL TAP - PETTY ($950 & UNDER)': 'Financial Crimes',
    'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT': 'Crimes Against Children',
    'TILL TAP - GRAND THEFT ($950.01 & OVER)': 'Financial Crimes',
    'HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE': 'Crimes Against Persons or Public Order',
    'FIREARMS RESTRAINING ORDER (FIREARMS RO)': 'Crimes Involving Weapons',
    'DRIVING WITHOUT OWNER CONSENT (DWOC)': 'Crimes Against Property and Vehicles',
    'DOCUMENT WORTHLESS ($200.01 & OVER)': 'Financial Crimes',
    'CRUELTY TO ANIMALS': 'Crimes Against Persons or Public Order',
    'CREDIT CARDS, FRAUD USE ($950.01 & OVER)': 'Financial Crimes',
    'LYNCHING - ATTEMPTED': 'Crimes Against Persons or Public Order',
    'FALSE POLICE REPORT': 'Crimes Against Persons or Public Order',
    'PANDERING': 'Crimes Against Persons or Public Order',
    'ILLEGAL DUMPING': 'Crimes Against Persons or Public Order',
    'VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, BIKES, ETC)': 'Crimes Against Property and Vehicles',
    'PETTY THEFT - AUTO REPAIR': 'Financial Crimes',
    'MANSLAUGHTER, NEGLIGENT': 'Violent Crimes',
    'RECKLESS DRIVING': 'Crimes Against Persons or Public Order',
    'PURSE SNATCHING - ATTEMPT': 'Property Crimes',
    'BIKE - ATTEMPTED STOLEN': 'Crimes Against Property and Vehicles',
    'CONSPIRACY': 'Crimes Against Persons or Public Order',
    'CONTRIBUTING': 'Crimes Against Persons or Public Order',
    'WEAPONS POSSESSION/BOMBING': 'Crimes Involving Weapons',
    'BRIBERY': 'Crimes Involving Weapons',
    'BOAT - STOLEN': 'Crimes Against Property and Vehicles',
    'THEFT, COIN MACHINE - GRAND ($950.01 & OVER)': 'Property Crimes',
    'GRAND THEFT / INSURANCE FRAUD': 'Financial Crimes',
    'LYNCHING': 'Crimes Against Persons or Public Order',
    'DISRUPT SCHOOL': 'Crimes Against Persons or Public Order',
    'DISHONEST EMPLOYEE - PETTY THEFT': 'Financial Crimes',
    'THEFT, COIN MACHINE - ATTEMPT': 'Property Crimes',
    'THEFT, COIN MACHINE - PETTY ($950 & UNDER)': 'Property Crimes',
    'REPLICA FIREARMS(SALE,DISPLAY,MANUFACTURE OR DISTRIBUTE)': 'Financial Crimes',
    'DRUGS, TO A MINOR': 'Financial Crimes',
    'GRAND THEFT / AUTO REPAIR': 'Financial Crimes',
    'PICKPOCKET, ATTEMPT': 'Property Crimes',
    'DRUNK ROLL': 'Crimes Against Persons or Public Order',
    'CHILD ABANDONMENT': 'Crimes Against Children',
    'TELEPHONE PROPERTY - DAMAGE': 'Crimes Against Persons or Public Order',
    'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM': 'Sex Crimes',
    'BIGAMY': 'Crimes Against Persons or Public Order',
    'FAILURE TO DISPERSE': 'Crimes Against Persons or Public Order',
    'FIREARMS EMERGENCY PROTECTIVE ORDER (FIREARMS EPO)': 'Crimes Involving Weapons',
    'INCEST (SEXUAL ACTS BETWEEN BLOOD RELATIVES)': 'Sexual Offenses',
    'BLOCKING DOOR INDUCTION CENTER': 'Crimes Against Persons or Public Order',
    'INCITING A RIOT': 'Crimes Against Persons or Public Order',
    'DISHONEST EMPLOYEE ATTEMPTED THEFT': 'Financial Crimes',
    'VANDALISM - MISDEAMEANOR ($399 OR UNDER)':'Crimes Against Property and Vehicles'
}

In [ ]:
data['Crime Category'] = data['Crm Cd Desc'].map(crime_category_dict)

In [ ]:
data['Crime Category'].unique()

In [ ]:
data['Crime Category'].value_counts()

In [ ]:
#Weapon Categorization (chatgpt)

In [ ]:
weapons_categories = {
    'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)': 'Physical Force',
    'UNKNOWN WEAPON/OTHER WEAPON': 'Unknown',
    'Unknown': 'Unknown',
    'ROCK/THROWN OBJECT': 'Melee',
    'VERBAL THREAT': 'Verbal Threat',
    'FOLDING KNIFE': 'Knife',
    'BLUNT INSTRUMENT': 'Melee',
    'BOTTLE': 'Melee',
    'SEMI-AUTOMATIC PISTOL': 'Firearm',
    'CLUB/BAT': 'Melee',
    'OTHER CUTTING INSTRUMENT': 'Knife',
    'HAND GUN': 'Firearm',
    'PHYSICAL PRESENCE': 'Physical Force',
    'VEHICLE': 'Vehicle',
    'SCISSORS': 'Knife',
    'STICK': 'Melee',
    'MACHETE': 'Melee',
    'OTHER KNIFE': 'Knife',
    'SHOTGUN': 'Firearm',
    'KNIFE WITH BLADE 6INCHES OR LESS': 'Knife',
    'FIRE': 'Fire',
    'ICE PICK': 'Melee',
    'GLASS': 'Melee',
    'SIMULATED GUN': 'Firearm',
    'KNIFE WITH BLADE OVER 6 INCHES IN LENGTH': 'Knife',
    'DEMAND NOTE': 'Threat Note',
    'BOMB THREAT': 'Threat Note',
    'PIPE/METAL PIPE': 'Melee',
    'UNKNOWN FIREARM': 'Firearm',
    'MACE/PEPPER SPRAY': 'Chemical Weapon',
    'HAMMER': 'Melee',
    'RAZOR': 'Knife',
    'OTHER FIREARM': 'Firearm',
    'BELT FLAILING INSTRUMENT/CHAIN': 'Melee',
    'UNKNOWN TYPE CUTTING INSTRUMENT': 'Knife',
    'SCREWDRIVER': 'Melee',
    'KITCHEN KNIFE': 'Knife',
    'AIR PISTOL/REVOLVER/RIFLE/BB GUN': 'Firearm',
    'BRASS KNUCKLES': 'Melee',
    'REVOLVER': 'Firearm',
    'SWITCH BLADE': 'Knife',
    'STUN GUN': 'Electric Weapon',
    'AXE': 'Melee',
    'RIFLE': 'Firearm',
    'ASSAULT WEAPON/UZI/AK47/ETC': 'Firearm',
    'ANTIQUE FIREARM': 'Firearm',
    'FIXED OBJECT': 'Melee',
    'SEMI-AUTOMATIC RIFLE': 'Firearm',
    'CAUSTIC CHEMICAL/POISON': 'Chemical Weapon',
    'TIRE IRON': 'Melee',
    'MARTIAL ARTS WEAPONS': 'Melee',
    'CONCRETE BLOCK/BRICK': 'Melee',
    'BOARD': 'Melee',
    'DIRK/DAGGER': 'Knife',
    'TOY GUN': 'Toy',
    'HECKLER & KOCH 93 SEMIAUTOMATIC ASSAULT RIFLE': 'Firearm',
    'MAC-11 SEMIAUTOMATIC ASSAULT WEAPON': 'Firearm',
    'EXPLOXIVE DEVICE': 'Explosive',
    'SAWED OFF RIFLE/SHOTGUN': 'Firearm',
    'DOG/ANIMAL (SIC ANIMAL ON)': 'Animal',
    'SYRINGE': 'Melee',
    'SCALDING LIQUID': 'Chemical Weapon',
    'RAZOR BLADE': 'Knife',
    'CLEAVER': 'Melee',
    'ROPE/LIGATURE': 'Melee',
    'BOW AND ARROW': 'Projectile',
    'AUTOMATIC WEAPON/SUB-MACHINE GUN': 'Firearm',
    'LIQUOR/DRUGS': 'Chemical Substance',
    'SWORD': 'Melee',
    'M1-1 SEMIAUTOMATIC ASSAULT RIFLE': 'Firearm',
    'STARTER PISTOL/REVOLVER': 'Firearm',
    'MAC-10 SEMIAUTOMATIC ASSAULT WEAPON': 'Firearm',
    'BOWIE KNIFE': 'Knife',
    'STRAIGHT RAZOR': 'Knife',
    'BLACKJACK': 'Melee',
    'RELIC FIREARM': 'Firearm',
    'HECKLER & KOCH 91 SEMIAUTOMATIC ASSAULT RIFLE': 'Firearm',
    'UZI SEMIAUTOMATIC ASSAULT RIFLE': 'Firearm',
    'UNK TYPE SEMIAUTOMATIC ASSAULT RIFLE': 'Firearm',
    'M-14 SEMIAUTOMATIC ASSAULT RIFLE': 'Firearm'
}


In [ ]:
data['Weapon Category'] = data['Weapon Desc'].map(weapons_categories)

In [ ]:
data['Weapon Category'].unique()

In [ ]:
#date

In [ ]:
data['Date Rptd'].dt.year.unique()

In [ ]:
data['Date Rptd Month'] = data['Date Rptd'].dt.strftime('%Y-%m')

In [ ]:
data['DATE OCC Month'] = data['DATE OCC'].dt.strftime('%Y-%m')

In [ ]:
data['Year'] = data['DATE OCC'].dt.year

In [ ]:
for x in ['Date Rptd Month', 'DATE OCC Month']:
    data[x] = pd.to_datetime(data[x])

In [ ]:
#TIME OCC catogery

In [ ]:
data['TIME OCC'].unique()

In [ ]:
def time(data):
    blanks = []
    for x in data['TIME OCC']:
        if 1 <= x <= 300:
            blanks.append('1AM - 3AM')
        elif 300 <= x <= 600:
            blanks.append('3AM - 6AM')
        elif 600 <= x <= 900:
            blanks.append('6AM - 9AM')
        elif 900 <= x <= 1200:
            blanks.append('9AM - 12PM')
        elif 1200 <= x <= 1500:
            blanks.append('12PM - 3PM')
        elif 1500 <= x <= 1800:
            blanks.append('3PM - 6PM')
        elif 1800 <= x <= 2100:
            blanks.append('6PM - 9PM')
        elif x >= 2100 or x <= 1200:
            blanks.append('9PM - 12AM')
    return blanks

In [ ]:
data['Time Period'] = time(data)

In [ ]:
data['Time Period'].unique()

In [ ]:
#General Visualization

In [ ]:
#Number of Crimes in Differenrt Area
fig = px.bar(
    data['AREA NAME'].value_counts().reset_index(), 
    x='index',
    y='AREA NAME',
    text_auto=True,
    color='index',
    title='Number of Crimes in Different Area Between 2020-01 and 2023-08',
    labels={'index':'Area'})

fig.show()

In [ ]:
#Number of Crimes in Different Crime Category
fig = px.bar(
    data['Crime Category'].value_counts().reset_index(), 
    x='index',
    y='Crime Category',
    text_auto=True,
    color='index',
    title='Number of Crimes in Different Crime Category Between 2020-01 and 2023-08',
    labels={'index':'Crime'})

fig.show()

In [ ]:
#Number of Crimes in Under Different Weapon Category
fig = px.pie(
    data['Weapon Category'].value_counts().reset_index(), 
    names='index',
    values='Weapon Category',
    title='Number of Crimes Under Different Weapon Category Between 2020-01 and 2023-08')\
    .update_traces(textposition='inside', textinfo='percent+label')

fig.show()

In [ ]:
#['Vict Descent','Vict Sex','Status Desc'] in data distribution

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Column Selection'),
    dcc.Dropdown(id='dropdown',
                options=['Vict Descent','Vict Sex','Status Desc'],
                value='Vict Descent'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(column):
    
    df = data[column].value_counts().reset_index()
    
    fig = px.pie(
        df, 
        names='index',
        values=column,
        title=f'{column} in Data Distribution'
    ).update_traces(textposition='inside', textinfo='percent+label')

    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9024)

In [ ]:
#['Date Rptd','DATE OCC'] year in data distribution
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Column Selection'),
    dcc.Dropdown(id='dropdown',
                options=['Date Rptd','DATE OCC'],
                value='Date Rptd'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(column):
    
    df = data[column].dt.year.value_counts().reset_index().sort_values('index')
    
    fig = px.pie(
        df, 
        names='index',
        values=column,
        title=f'Year Distribution of {column} in Data',
    ).update_traces(textposition='inside', textinfo='percent+label')

    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9025)

In [ ]:
#Number of Crimes in different time period
fig = px.bar(
    data['Time Period'].value_counts().reset_index(), 
    x='index',
    y='Time Period',
    color='index',
    text_auto=True,
    title='Number of Crimes in Different Time Period Between 2020-01 and 2023-08')

fig.show()


In [ ]:
#Crimes Cnt in Different Year
month = data['DATE OCC Month'].value_counts().reset_index().sort_values('index')
month['Year'] = month['index'].dt.year

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection'),
    dcc.Dropdown(id='dropdown',
                options=month.Year.unique(),
                value=2020,
#                 multi=True
                ),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), [Input('dropdown', 'value')])
def plot(year):
#     month = data['DATE OCC Month'].value_counts().reset_index().sort_values('index')
#     month['Year'] = month['index'].dt.year

    fig = px.line(
        month.query("Year == @year"), 
        x='index',
        y='DATE OCC Month',
        color='Year',
        labels={'DATE OCC Month':'Crimes Cnt',
               'index':'Date of Occurrence'},
        title=f'Crimes Cnt in {year}',
        markers=True
    )

    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9026)

In [ ]:
month['Month'] = month['index'].dt.strftime('%m')

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection'),
    dcc.Dropdown(id='dropdown',
                options=month.Year.unique(),
                value=2020,
                multi=True),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), [Input('dropdown', 'value')])
def plot(year):

    fig = px.line(
        month.query("Year == @year"), 
        x='Month',
        y='DATE OCC Month',
        color='Year',
        labels={'DATE OCC Month':'Crimes Cnt',
               'index':'Date of Occurrence'},
        title=f'Crimes Cnt in {year}',
        markers=True
    )
    
#     custom_x_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    fig.update_xaxes(tickvals=list(range(0, 13)))

    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9027)

In [ ]:
Map = data.loc[data['LAT'] != 0.0000]

In [ ]:
df = Map.groupby(['LAT', 'LON']).size().reset_index(name='Count')
df

In [ ]:
fig = px.scatter_mapbox(df, lat="LAT", lon="LON", 
                        color="Count", 
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=15, zoom=10)

fig.show()

Aera Name Analysis 

In [ ]:
#Crime Counts in Each AREA
px.bar(
    data['AREA NAME'].value_counts().reset_index(),
    x='index',
    y='AREA NAME',
    color='index',
    title='Crime Counts in Each Area',
    text_auto=True,
    labels={'AREA NAME':'Cnt', 'index':'Area'})

In [ ]:
#Avg Age of Victims in Different Areas
px.bar(
    data.groupby('AREA NAME')['Vict Age'].mean().sort_values(ascending=False).reset_index(),
    x='AREA NAME',
    y='Vict Age',
    color='AREA NAME',
    text_auto=True,
    title ='Avg Age of Victims in Different Areas'
)


In [ ]:
#Avg Age of Victims in Different Areas in Different Years
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown',
                options=[2020, 2021, 2022, 2023],
                value=2020),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown', 'value'))
def plot(year):
    df = pd.DataFrame(data.groupby(['Year','AREA NAME'])['Vict Age'].mean()).reset_index()
    
    fig=px.bar(
        df.loc[df['Year']==year].sort_values('Vict Age', ascending=False),
        x='AREA NAME',
        y='Vict Age',
        color='AREA NAME'
    ).add_hline(
          y=df['Vict Age'].mean(),
          line_dash='dash',
          line_color='grey')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9028)

In [ ]:
#Avg Age of Different Gender in Different Area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown', 'value'))
def plot(loc):
    df = pd.DataFrame(data.groupby(['AREA NAME', 'Vict Sex'])['Vict Age'].mean()).reset_index()
    
    fig=px.bar(
        df.loc[df['AREA NAME']==loc],
        x='Vict Sex',
        y='Vict Age',
        color='Vict Sex',
        text_auto=True,
        title=f"Avg Age of Different Gender in {loc}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9029)

In [ ]:
#Avg Age of Different Gender in Different Area
pd.DataFrame(data.groupby(['AREA NAME', 'Vict Sex'])['Vict Age'].mean()).reset_index()

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=data['Year'].unique(),
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year', 'AREA NAME', 'Vict Sex'])['Vict Age'].mean()).reset_index()
    
    fig=px.bar(
        df.loc[(df['Year']==loc)&(df['AREA NAME']==year)],
        x='Vict Sex',
        y='Vict Age',
        color='Vict Sex',
        text_auto=True,
        title=f"Avg Age of Different Gender in {year} in {loc}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9030)

In [ ]:
#Avg Age of Different Gender in Different Area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Area Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year', 'AREA NAME', 'Vict Descent'])['Vict Age'].mean()).reset_index()
    
    df['Vict Age'] = round(df['Vict Age'],1)
    
    fig=px.bar(
        df.loc[(df['Year']==loc)&(df['AREA NAME']==year)].sort_values('Vict Age', ascending=False),
        x='Vict Descent',
        y='Vict Age',
        color='Vict Descent',
        text_auto=True,
        title=f"Avg Age of Different Victim Descent in {year} in {loc}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9034)

In [ ]:
#Victims Gender Cnt in Different Area 
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area Selection:'),
    dcc.Dropdown(id='dropdown',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown', 'value'))
def plot(loc):
    df = pd.DataFrame(data.groupby('AREA NAME')['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig=px.pie(
        df.loc[df['AREA NAME']==loc],
        names='Vict Sex',
        values='Cnt',
        title=f"Victim Gender Cnt at {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9031)

In [ ]:
#Victims Gender Cnt in Different Area in different Year
pd.DataFrame(data.groupby(['AREA NAME', 'Vict Sex'])['Vict Age'].mean()).reset_index()

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year', 'AREA NAME'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig=px.pie(
        df.loc[(df['Year']==loc)&(df['AREA NAME']==year)],
        names='Vict Sex',
        values='Cnt',
        title=f"Victim Gender Cnt at {year} in {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9032)

In [ ]:
#Victim Descent Cnt at different Area name in different year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year', 'AREA NAME'])['Vict Descent'].value_counts()).rename(columns={'Vict Descent':'Cnt'}).reset_index()
    
    fig=px.pie(
        df.loc[(df['Year']==loc)&(df['AREA NAME']==year)],
        names='Vict Descent',
        values='Cnt',
        title=f"Victim Descent Cnt at {year} in {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9033)

In [ ]:
#Crime Category Cnt at Different Location in Different Year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Area Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year', 'AREA NAME'])['Crime Category'].value_counts()).rename(columns={'Crime Category':'Cnt'}).reset_index()
    
    fig=px.bar(
        df.loc[(df['Year']==loc)&(df['AREA NAME']==year)],
        x='Crime Category',
        y='Cnt',
        color='Crime Category',
        text_auto=True,
        title=f"Crime Category Cnt at {year} in {loc}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9035)

In [ ]:
#Avg Victim Age in each area in Different Crime Category
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area Selection:'),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'))
def plot(loc):
    df = pd.DataFrame(data.groupby(['AREA NAME','Crime Category'])['Vict Age'].mean()).reset_index().sort_values('Vict Age', ascending=False)
    df['Vict Age'] = round(df['Vict Age'], 1)
    fig=px.bar(
        df.loc[df['AREA NAME']==loc],
        x='Crime Category',
        y='Vict Age',
        color='Crime Category',
        text_auto=True,
        title=f"Avg Victim Age in {loc} in Different Crime Category"
    ).update_traces(showlegend=False)
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9037)

In [ ]:
#Gender Distribution for Each Crime Category
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Crime Selection:'),
    dcc.Dropdown(id='dropdown1',
                options=data['Crime Category'].unique(),
                value='Violent Crimes'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'))
def plot(loc):
    df = pd.DataFrame(data.groupby(['Crime Category'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()

    fig=px.pie(
        df.loc[df['Crime Category']==loc],
        names='Vict Sex',
        values='Cnt',
        title=f"Gender Distribution for {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')

    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9038)

In [ ]:
#Gender Distribution for Different Crimes in each Area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area / Crime Selection:'),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    html.Br(),
    dcc.Dropdown(id='dropdown2',
                options=data['Crime Category'].unique(),
                value='Violent Crimes'), 
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(loc, crime):
    df = pd.DataFrame(data.groupby(['AREA NAME','Crime Category'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig=px.pie(
        df.loc[(df['AREA NAME']==loc) & (df['Crime Category']==crime)],
        names='Vict Sex',
        values='Cnt',
        title=f"Gender Distribution for {crime} in {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9039)

In [ ]:
#Each Weapon Category Cnt at Area in Year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Area Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year', 'AREA NAME'])['Weapon Category'].value_counts())\
         .rename(columns={'Weapon Category':'Cnt'}).reset_index()
    
    fig=px.bar(
        df.loc[(df['Year']==loc)&(df['AREA NAME']==year)],
        x='Weapon Category',
        y='Cnt',
        color='Weapon Category',
        text_auto=True,
        title=f"Each Weapon Category Cnt at {year} in {loc}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9040)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(area):
    df = pd.DataFrame(data.groupby('DATE OCC Month')['AREA NAME'].value_counts())\
            .rename(columns={'AREA NAME':'Cnt'}).reset_index()
    
    fig = px.line(
        df.loc[df['AREA NAME']==area],
        x='DATE OCC Month',
        y='Cnt', 
        markers=True,
        labels={'DATE OCC Month':'Date', 'Cnt':'Crimes Cnt'},
        title=f'Total Crime Cnt in each Month at {area} between 2020-01 and 2023-08'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9041)

In [ ]:
#Total Crime Cnt in different years in Different Areas
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=[2020,2021,2022,2023],
                value=2020),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(year):
    df = pd.DataFrame(data.groupby('Year')['AREA NAME'].value_counts())\
        .rename(columns={'AREA NAME':'Cnt'}).reset_index()
    
    fig = px.bar(
        df.loc[df['Year']==year],
        x='AREA NAME',
        y='Cnt', 
        labels={'Cnt':'Crimes Cnt'},
        color='AREA NAME',
        text_auto=True,
        title=f'Total Crime Cnt in {year} in Different Areas'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9042)

In [ ]:
#Total Crime Cnt in Different Time Period in different area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(area):
    df=pd.DataFrame(data.groupby('AREA NAME')['Time Period'].value_counts())\
                .rename(columns={'Time Period':'Cnt'}).reset_index()
    
    fig = px.bar(
        df.loc[df['AREA NAME']==area],
        x='Time Period',
        y='Cnt', 
#         labels={'Cnt':'Crimes Cnt'},
        color='Time Period',
        text_auto=True,
        title=f'Total Crime Cnt in Different Time Period in {area}'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9043)

Victim Age Analysis

In [ ]:
px.histogram(
    data, 
    x='Vict Age',
    nbins=30, 
    labels={'value':'age'},
    color='Vict Sex'
)

In [ ]:
#boxplot
px.box(
    data, 
    y='Vict Age'
)

In [ ]:
data.loc[data['Vict Age'] == 120]

In [ ]:
data.loc[data['Vict Sex'] == 'Unknown']['Vict Age'].value_counts()
#The majority of the victims whose sex is unknown has the age of 30.

In [ ]:
#Avg Victims Age in Different Gender
px.bar(
    data.groupby('Vict Sex')['Vict Age'].mean().reset_index().sort_values('Vict Age', ascending=False),
    x='Vict Sex',
    y='Vict Age',
    color='Vict Sex',
    text_auto=True,
    title='Avg Victims Age in Different Gender'
)

In [ ]:
vict_desc_avg_age =\
data.groupby(['Vict Descent'])['Vict Age'].mean().reset_index().sort_values('Vict Age', ascending=False)

vict_desc_avg_age['Vict Age'] = round(vict_desc_avg_age['Vict Age'], 1)

In [ ]:
#Avg Victims Age of Different Ethnicity
px.bar(
    vict_desc_avg_age,
    x='Vict Descent',
    y='Vict Age',
    color='Vict Descent',
    text_auto=True,
    title='Avg Victims Age of Different Ethnicity'
)

In [ ]:
#Avg Victims Age of Each Gender in different ethinicity

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Ethnicity Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['Vict Descent'].unique(),
                value='Asian'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(ethi):
    df=data.groupby(['Vict Descent', 'Vict Sex'])['Vict Age'].mean().reset_index()
    
    fig = px.bar(
        df.loc[df['Vict Descent']==ethi],
        x='Vict Sex',
        y='Vict Age', 
#         labels={'Cnt':'Crimes Cnt'},
        color='Vict Sex',
        text_auto=True,
        title=f'Avg Victims Age of Each Gender in {ethi}'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=9044)

In [ ]:
#Avg Age of Each Gender in different year among Different Gender in each ethnicity"
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Area Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['Vict Descent'].unique(),
                value='Asian'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, ethi):
    df = data.groupby(['Year', 'Vict Descent', 'Vict Sex'])['Vict Age'].mean().reset_index()
    
    fig=px.bar(
        df.loc[(df['Year']==ethi)&(df['Vict Descent']==year)],
        x='Vict Sex',
        y='Vict Age',
        color='Vict Sex',
        text_auto=True,
        title=f"Avg Age of Each Gender in {year} among Different Gender in {ethi}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8001)

In [ ]:
#Avg Victim Age of Each Crime Category
descent_age = data.groupby(['Crime Category'])['Vict Age'].mean().reset_index().sort_values('Vict Age', ascending=False)
descent_age['Vict Age'] = round(descent_age['Vict Age'],1)

px.bar(
    descent_age,
    x='Crime Category',
    y='Vict Age',
    text_auto=True,
    color='Crime Category',
    title='Avg Victim Age of Each Crime Category'
).update_traces(showlegend=False)

In [ ]:
#Avg Victim Age in each area in Different Crime Category
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Area Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = data.groupby(['Year','AREA NAME','Crime Category'])['Vict Age'].mean().reset_index()
    df['Vict Age'] = round(df['Vict Age'], 1)
    
    fig=px.bar(
        df.loc[(df['Year']==loc)&(df['AREA NAME']==year)].sort_values('Vict Age', ascending=False),
        x='Crime Category',
        y='Vict Age',
        color='Crime Category',
        text_auto=True,
        title=f"Avg Age in {year} in {loc} of Different Crime Category"
    ).update_traces(showlegend=False)
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8002)

In [ ]:
px.bar(
    data.groupby('Weapon Category')['Vict Age'].mean().reset_index().sort_values('Vict Age', ascending=False),
    x='Weapon Category',
    y='Vict Age',
    color='Weapon Category',
    title='Avg Victim Age of Attacked by Different Weapon Category'
).add_hline(
          y=data['Vict Age'].mean(),
          line_dash='dash',
          line_color='grey')


In [ ]:
#Avg Victims Age of Each Gender in different Area under Different Weapon Category
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(loc):
    df=data.groupby(['AREA NAME','Weapon Category'])['Vict Age'].mean().reset_index()
    df['Vict Age'] = round(data['Vict Age'],1)
    
    fig = px.bar(
        df.loc[df['AREA NAME']==loc].sort_values('Vict Age', ascending=False),
        x='Weapon Category',
        y='Vict Age', 
#         labels={'Cnt':'Crimes Cnt'},
        color='Weapon Category',
        text_auto=True,
        title=f'Avg Victims Age in {loc} Attacked by Different Weapon Category'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8003)

In [ ]:
#Avg Victims Age of Each Gender in Different Area in differrent Year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(year):
    df=data.groupby(['Year','AREA NAME'])['Vict Age'].mean().reset_index()
    df['Vict Age'] = round(data['Vict Age'],1)
    
    fig = px.bar(
        df.loc[df['Year']==year].sort_values('Vict Age', ascending=False),
        x='AREA NAME',
        y='Vict Age', 
#         labels={'Cnt':'Crimes Cnt'},
        color='AREA NAME',
        text_auto=True,
        title=f'Avg Victims Age of Each Gender in Different Area in {year}'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8004)

Victim Sex Analysis

In [ ]:
#Gender Distribution
px.pie(
    data['Vict Sex'].value_counts().reset_index(),
    names='index',
    values='Vict Sex',
    title='Gender Distribution'
).update_traces(textposition='inside', textinfo='percent+label')

In [ ]:
#Victim Gender Distribution Among different ethnicity
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Ethnicity Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['Vict Descent'].unique(),
                value='Asian'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(ethi):
    df=pd.DataFrame(data.groupby('Vict Descent')['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig = px.pie(
        df.loc[df['Vict Descent']==ethi],
        names='Vict Sex',
        values='Cnt', 
#         labels={'Cnt':'Crimes Cnt'},
        title=f'Victim Gender Distribution Among {ethi}'
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8005)

In [ ]:
#Victim Gender Distribution among Different ethnicity in different Year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Area Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['Vict Descent'].unique(),
                value='Asian'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year','Vict Descent'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig=px.pie(
        df.loc[(df['Year']==loc)&(df['Vict Descent']==year)].sort_values('Cnt', ascending=False),
        names='Vict Sex',
        values='Cnt',
        title=f"Victim Gender Distribution among {year} in {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8006)

In [ ]:
#Victim Gender Distribution in Different Crime Category
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Crime Category Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['Crime Category'].unique(),
                value='Crimes Against Children'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(crime):
    df=pd.DataFrame(data.groupby(['Crime Category'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig = px.pie(
        df.loc[df['Crime Category']==crime],
        names='Vict Sex',
        values='Cnt', 
#         labels={'Cnt':'Crimes Cnt'},
        title=f'Victim Gender Distribution in {crime}'
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8007)


In [ ]:
#Victim Gender Distribution in Different Crime Category in each Year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Crime Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['Crime Category'].unique(),
                value='Crimes Against Children'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year','Crime Category'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig=px.pie(
        df.loc[(df['Year']==loc)&(df['Crime Category']==year)],
        names='Vict Sex',
        values='Cnt',
        title=f"Victim Gender Distribution in {year} in {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8008)

In [ ]:
#Victim Gender Distribution Attacked by Weapon category in each year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Crime Selection:'),
    dcc.Dropdown(id='dropdown2',
                options=[2020, 2021, 2022, 2023],
                value=2020), 
    html.Br(),
    dcc.Dropdown(id='dropdown1',
                options=data['Weapon Category'].unique(),
                value='Physical Force'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual','figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, loc):
    df = pd.DataFrame(data.groupby(['Year','Weapon Category'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig=px.pie(
        df.loc[(df['Year']==loc)&(df['Weapon Category']==year)],
        names='Vict Sex',
        values='Cnt',
        title=f"Victim Gender Distribution Attacked by {year} in {loc}"
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8009)

In [ ]:
#Victim Gender Distribution in each Year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(year):
    df=pd.DataFrame(data.groupby(['Year'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig = px.pie(
        df.loc[df['Year']==year],
        names='Vict Sex',
        values='Cnt', 
#         labels={'Cnt':'Crimes Cnt'},
        title=f'Victim Gender Distribution in {year}'
    ).update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8010)


In [ ]:
#Victim Gender value_counts over the years
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(year):
    df=pd.DataFrame(data.groupby(['DATE OCC Month'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()
    
    fig = px.line(
    df.loc[df['DATE OCC Month'].dt.year == year],
    x='DATE OCC Month',
    y='Cnt',
    color='Vict Sex',
    markers=True, 
    title=f'Victim Gender Value Counts in {year} in each Month'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8011)

In [ ]:
#Victim Gender value_counts over the years in differnet location 
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Area Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    html.Br(),
    dcc.Dropdown(id='dropdown2', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, area):
    df=pd.DataFrame(data.groupby(['AREA NAME','DATE OCC Month'])['Vict Sex'].value_counts()).rename(columns={'Vict Sex':'Cnt'}).reset_index()

    fig = px.line(
    df.loc[(df['DATE OCC Month'].dt.year == year) & (df['AREA NAME'] == area)],
    x='DATE OCC Month',
    y='Cnt',
    color='Vict Sex',
    markers=True, 
    title=f'Victim Gender Value Counts in {year} in {area}'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8012)

Vict Descent Analysis 

In [ ]:
#Victim Ethnicity Distribution
px.bar(
    data['Vict Descent'].value_counts().reset_index(), 
    x='index', 
    y='Vict Descent',
    text_auto=True,
    color='index',
    title='Victim Ethnicity Distribution'
)

In [ ]:
#Victim Ethnicity Value Counts in each area in different years'
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year/Area Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    html.Br(),
    dcc.Dropdown(id='dropdown2', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, area):
    df=pd.DataFrame(data.groupby(['Year','AREA NAME'])['Vict Descent'].value_counts())\
        .rename(columns={'Vict Descent':'Cnt'}).reset_index()
    
    fig = px.bar(
    df.loc[(df['Year'] == year) & (df['AREA NAME'] == area)],
    x='Vict Descent',
    y='Cnt',
    color='Vict Descent',
    text_auto=True,
    title=f'Victim Ethnicity Value Counts in {area} in {year}'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8013)

In [ ]:
#Avg Victim Ethnicity's Age in different area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area Selection:'),
    dcc.Dropdown(id='dropdown2', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown2', 'value'))
def plot(area):
    df=data.groupby(['AREA NAME', 'Vict Descent'])['Vict Age'].mean().reset_index()
    df['Vict Age'] = round(df['Vict Age'], 1)
    
    fig = px.bar(
    df.loc[df['AREA NAME'] == area].sort_values('Vict Age', ascending=False),
    x='Vict Descent',
    y='Vict Age',
    color='Vict Descent',
    text_auto=True,
    title=f"Avg Victim Ethnicity's Age in {area}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8014)

In [ ]:
#Victim Ethnicity Count in crime category in a area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area/Crime Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    html.Br(),
    dcc.Dropdown(id='dropdown2', 
                options=data['Crime Category'].unique(),
                value='Crimes Against Children'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'),Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(area, crime):
    df=pd.DataFrame(data.groupby(['AREA NAME', 'Crime Category'])['Vict Descent'].value_counts())\
    .rename(columns={'Vict Descent':'Cnt'}).reset_index()
    
    fig = px.bar(
    df.loc[(df['AREA NAME'] == area)&(df['Crime Category'] == crime)].sort_values('Cnt', ascending=False),
    x='Vict Descent',
    y='Cnt',
    color='Vict Descent',
    text_auto=True,
    title=f"Victim Ethnicity Count in {crime} in {area}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8015)

In [ ]:
#Victim Ethnicity Count attacked by in a weapon category in a area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area/Weapon Category Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    html.Br(),
    dcc.Dropdown(id='dropdown2', 
                options=data['Weapon Category'].unique(),
                value='Physical Force'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'),Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(area, weapon):
    df=pd.DataFrame(data.groupby(['AREA NAME','Weapon Category'])['Vict Descent'].value_counts())\
    .rename(columns={'Vict Descent':'Cnt'}).reset_index()
    
    fig = px.bar(
    df.loc[(df['AREA NAME'] == area)&(df['Weapon Category'] == weapon)].sort_values('Cnt', ascending=False),
    x='Vict Descent',
    y='Cnt',
    color='Vict Descent',
    text_auto=True,
    title=f"Victim Ethnicity Count attacked by {weapon} in {area}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8016)

In [ ]:
#Victim Ethnicity Count in Different Year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(year):
    df=pd.DataFrame(data.groupby('Year')['Vict Descent'].value_counts())\
    .rename(columns={'Vict Descent':'Cnt'}).reset_index()
    
    fig = px.bar(
    df.loc[df['Year']== year],
    x='Vict Descent',
    y='Cnt',
    color='Vict Descent',
    title=f'Victim Ethnicity Count in {year}',
    text_auto=True
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8017)

In [ ]:
#Victim Ethnicity Count in each Month
px.line(
    pd.DataFrame(data.groupby('DATE OCC Month')['Vict Descent'].value_counts())\
    .rename(columns={'Vict Descent':'Cnt'}).reset_index(),
    x='DATE OCC Month',
    y='Cnt',
    color='Vict Descent',
    title='Victim Ethnicity Count in each Month'
)

In [ ]:
#Different Ethnicity Victim Count over the Years
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Ethnicity Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['Vict Descent'].unique(),
                value='Hispanic/Latin/Mexican'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(ethi):
    df = pd.DataFrame(data.groupby('DATE OCC Month')['Vict Descent'].value_counts())\
    .rename(columns={'Vict Descent':'Cnt'}).reset_index()
    
    fig = px.line(
    df.loc[df['Vict Descent']== ethi],
    x='DATE OCC Month',
    y='Cnt',
    markers=True,
    title=f'{ethi} Victim Count over the Years'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8018)

Status Desc Analysis

In [ ]:
data['Status Desc'].unique()

In [ ]:
#Status Desc in data Distribution
px.pie(
    data['Status Desc'].value_counts().reset_index(), 
    names='index',
    values='Status Desc',
    title='Status Desc in data Distribution'
).update_traces(textposition='inside', textinfo='percent+label')

In [ ]:
#Crime Desc in different Crime Category in each year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Crime Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    html.Br(),
    dcc.Dropdown(id='dropdown2', 
                options=data['Crime Category'].unique(),
                value='Crimes Against Children'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, crime):
    df=pd.DataFrame(data.groupby(['Year', 'Crime Category'])['Status Desc'].value_counts())\
    .rename(columns={'Status Desc':'Cnt'}).reset_index()
    
    fig = px.bar(
    df.loc[(df['Year']== year) & (df['Crime Category']== crime)],
    x='Status Desc',
    y='Cnt',
    color='Status Desc',
    title=f'Status in {crime} in {year}',
    text_auto=True
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8020)

In [ ]:
#Crime Desc in different weapon Category in each year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Weapon Category Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    html.Br(),
    dcc.Dropdown(id='dropdown2', 
                options=data['Weapon Category'].unique(),
                value='Physical Force'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, weapon):
    df=pd.DataFrame(data.groupby(['Year', 'Weapon Category'])['Status Desc'].value_counts())\
    .rename(columns={'Status Desc':'Cnt'}).reset_index()
    
    fig = px.bar(
    df.loc[(df['Year']== year) & (df['Weapon Category']== weapon)],
    x='Status Desc',
    y='Cnt',
    color='Status Desc',
    title=f'Status Count in {weapon} in {year}',
    text_auto=True
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8021)

In [ ]:
#Crime Desc in different area in each year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year / Weapon Category Selection:'),
    dcc.Dropdown(id='dropdown1', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    html.Br(),
    dcc.Dropdown(id='dropdown2', 
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(year, area):
    df=pd.DataFrame(data.groupby(['Year', 'AREA NAME'])['Status Desc'].value_counts())\
    .rename(columns={'Status Desc':'Cnt'}).reset_index()
    
    fig = px.bar(
    df.loc[(df['Year']== year) & (df['AREA NAME']== area)],
    x='Status Desc',
    y='Cnt',
    color='Status Desc',
    title=f'Status Count in {area} in {year}',
    text_auto=True
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8022)

In [ ]:
#Status Desc Count over the Years
px.line(
    pd.DataFrame(data.groupby('DATE OCC Month')['Status Desc'].value_counts())\
.rename(columns={'Status Desc':'Cnt'}).reset_index(),
    x='DATE OCC Month',
    y='Cnt',
    color='Status Desc',
    title='Status Count over the Years'
)

In [ ]:
#Status Count Over the years 
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Status Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['Status Desc'].unique(),
                value='Invest Cont'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(status):
    df = pd.DataFrame(data.groupby('DATE OCC Month')['Status Desc'].value_counts())\
    .rename(columns={'Status Desc':'Cnt'}).reset_index()
    
    fig = px.line(
    df.loc[df['Status Desc']== status],
    x='DATE OCC Month',
    y='Cnt',
    markers=True,
    title=f'{status} Count over the Years'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=6001)

Crime Category Analysis

In [ ]:
data['Crime Category'].unique()

In [ ]:
##Crime Category Distribution
px.bar(
    data['Crime Category'].value_counts().reset_index(),
    x='index',
    y='Crime Category',
    color='index',
    title='Crime Category Distribution',
    text_auto=True,
    labels={'index':'Crime Category', 'Crime Category':'Cnt'}
)

In [ ]:
#Count of Weapon Category used in certain crime
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Crime Selection:'),
    dcc.Dropdown(id='dropdown2', 
                options=data['Crime Category'].unique(),
                value='Crimes Against Children'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown2', 'value'))
def plot(crime):
    df=pd.DataFrame(data.groupby('Crime Category')['Weapon Category'].value_counts())\
    .rename(columns={'Weapon Category':'Cnt'}).reset_index()

    fig = px.bar(
    df.loc[df['Crime Category'] == crime],
    x='Weapon Category',
    y='Cnt',
    color='Weapon Category',
    text_auto=True,
    title=f"Count of Weapon Category used in {crime}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=6002)

In [ ]:
#Crime Category Count in different year
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Year Selection:'),
    dcc.Dropdown(id='dropdown2', 
                options=[2020, 2021, 2022, 2023],
                value=2020),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown2', 'value'))
def plot(year):
    df=pd.DataFrame(data.groupby('Year')['Crime Category'].value_counts())\
    .rename(columns={'Crime Category':'Cnt'}).reset_index()

    fig = px.bar(
    df.loc[df['Year'] == year],
    x='Crime Category',
    y='Cnt',
    color='Crime Category',
    text_auto=True,
    title=f"Crime Category Count in {year}"
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=6003)

In [ ]:
#Crimes Count for each Crime Category 
px.line(
    pd.DataFrame(data.groupby('DATE OCC Month')['Crime Category'].value_counts())\
    .rename(columns={'Crime Category':'Cnt'}).reset_index(),
    x='DATE OCC Month',
    y='Cnt',
    color='Crime Category',
    markers=True,
    title='Crimes Count for each Crime Category between 2020-01 and 2023-08')

In [ ]:
#Crimes Count over the years
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Crime Category Selection:'),
    dcc.Dropdown(id='dropdown', 
                options=data['Crime Category'].unique(),
                value='Property Crimes'),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def plot(crime):
    df = pd.DataFrame(data.groupby('DATE OCC Month')['Crime Category'].value_counts())\
    .rename(columns={'Crime Category':'Cnt'}).reset_index()
    
    fig = px.line(
    df.loc[df['Crime Category']== crime],
    x='DATE OCC Month',
    y='Cnt',
    markers=True,
    title=f'{crime} Count over the Years'
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=6004)

In [ ]:
#Number of Crime Count between timeperiod in a area
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Area / Time Period Selection:'),
    dcc.Dropdown(id='dropdown1',
                options=data['AREA NAME'].unique(),
                value='77th Street'),
    html.Br(),
    dcc.Dropdown(id='dropdown2',
                options=data['Time Period'].unique(),
                value='12PM - 3PM'),
    dcc.Graph(id='visual')
])
@app.callback(Output('visual', 'figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'))
def plot(area, timeperiod):
    df = pd.DataFrame(data.groupby(['AREA NAME','Time Period'])['Crime Category'].value_counts())\
    .rename(columns={'Crime Category':'Cnt'}).reset_index()
    
    fig=px.bar(
        df.loc[(df['AREA NAME']==area)&(df['Time Period']==timeperiod)],
        x='Crime Category',
        y='Cnt',
        color='Crime Category',
        text_auto=True,
        title=f'Number of Crime Count between {timeperiod} in {area}'
    )

    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=6005)

Correlation

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
dfm = data.copy()

In [ ]:
for x in ['AREA NAME', 'Vict Sex', 'Vict Descent', 'Status Desc', 
          'Crime Category', 'Weapon Category', 'Time Period']:
    dfm[x] = label_encoder.fit_transform(dfm[x])

In [ ]:
df = dfm.loc[:, ['TIME OCC','AREA','Vict Age', 'Vict Sex', 'Vict Descent', 'Status Desc',
           'LAT', 'LON', 'Crime Category', 'Weapon Category', 'Year', 'Time Period']]

In [ ]:
df.head()

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True)


In [ ]:
#Scatterplot 
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Column Selection'),
    dcc.Dropdown(id='dropdown1',
                options=df.corr().columns,
                value='TIME OCC'),
    html.Br(),
    dcc.Dropdown(id='dropdown2',
                options=df.corr().columns,
                value='Vict Sex'),
    html.Br(),
    dcc.RadioItems(id='dropdown3',
                options=['Trendline On', 'Trendline Off'],
                value='Trendline off'),
    dcc.Graph(id='visual')
])
@app.callback(Output('visual', 'figure'), Input('dropdown1', 'value'), Input('dropdown2', 'value'),
             Input('dropdown3', 'value'))
def plot(x, y, switch):
    
    fig=px.scatter(
        df, 
        x=x,
        y=y,
        trendline=None if switch == 'Trendline Off' else 'ols'
    
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=6006)

Prediction on Vict Gender

In [ ]:
columns = [x for x in df.columns if x!='Vict Descent']

In [ ]:
for col in columns:
    dict1 = df.groupby([col])['Vict Sex'].mean().to_dict()
    df[col]=df[col].map(dict1)

In [ ]:
df.head()

In [ ]:
df.corr()['Vict Sex'].sort_values(ascending=False)

In [ ]:
X = df.drop('Vict Sex', axis=1)

y = df.loc[:, 'Vict Sex']

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [ ]:
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score

In [ ]:
model = SelectFromModel(Lasso(alpha=0.005))

In [ ]:
model.fit(X, y)

In [ ]:
model.get_support()

In [ ]:
cols = X.columns[model.get_support()]

In [ ]:
X=X[cols]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(X_train, y_train)

In [ ]:
pred = log_reg.predict(X_test)

In [ ]:
mean_squared_error(pred, y_test)

In [ ]:
confusion_matrix(pred, y_test)

In [ ]:
accuracy_score(pred, y_test)

In [ ]:
def display_scores(scores):
    print('===============================================')
    print('Scores: {}'.format(scores))
    print('===============================================')
    print('Mean Score: {}'.format(scores.mean()))
    print('===============================================')
    print('Standard Deviation of Scores: {}'.format(scores.std()))
    print('===============================================')
    
    return None

In [ ]:
scores = cross_val_score(log_reg,
               X_train,
               y_train,
#                scoring='neg_mean_squared_error',
               cv=10)

In [ ]:
display_scores(scores)

In [ ]:
plt.figure()
sns.kdeplot(pred, shade=True)
sns.kdeplot(y_test, shade=True)
plt.legend(['Pred', 'y_test'])

In [ ]:
#DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier()

In [ ]:
tree.fit(X_train, y_train)

In [ ]:
pred = tree.predict(X_test)

In [ ]:
accuracy_score(pred, y_test)

In [ ]:
mean_squared_error(pred, y_test)

In [ ]:
confusion_matrix(pred, y_test)

In [ ]:
plt.figure()
sns.kdeplot(pred, shade=True)
sns.kdeplot(y_test, shade=True)
plt.legend(['Pred', 'y_test'])

In [ ]:
#Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier()

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
pred = forest.predict(X_test)

In [ ]:
mean_squared_error(pred, y_test)

In [ ]:
accuracy_score(pred, y_test)

In [ ]:
confusion_matrix(pred, y_test)

In [ ]:
plt.figure()
sns.kdeplot(pred, shade=True)
sns.kdeplot(y_test, shade=True)
plt.legend(['Pred', 'y_test'])